## Julia Docker images

Available at [dockerhub](https://hub.docker.com/_/julia)

`docker pull julia:1.4.1`

In [1]:
run(pipeline(`docker images`, `grep julia`, `grep "1\.4\.1"`));

julia                                1.4.1               248f01656dfd        13 days ago         447MB


## Run a simple Julia Task

Let us first run a Julia task that just prints out something

In [2]:
using Kuber
ctx = KuberContext();
Kuber.set_api_versions!(ctx);

┌ Info: unsupported crd.projectcalico.org/v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:131


In [3]:
julia_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "julia-pod",
               "namespace": "default",
               "labels": {
                   "name": "julia-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "julia",
                   "image": "julia:1.4.1",
                   "command": ["julia"],
                   "args": [
                       "-e",
                       "while true; println(time()); sleep(1); end"
                   ]
               }]
           }
       }""");

In [4]:
pod = put!(ctx, julia_pod);

In [5]:
;kubectl logs julia-pod

1.588797586431455e9
1.588797587596682e9
1.588797588598617e9
1.588797589600988e9
1.588797590602545e9
1.588797591604946e9
1.588797592606664e9
1.588797593608763e9
1.588797594610913e9
1.588797595613233e9
1.588797596615522e9
1.588797597618034e9
1.588797598620108e9
1.588797599622294e9
1.588797600624468e9
1.588797601626633e9
1.588797602629084e9
1.588797603630638e9
1.588797604633087e9
1.588797605635429e9
1.588797606637601e9
1.588797607638972e9
1.588797608641356e9
1.588797609642598e9
1.588797610644727e9
1.588797611646566e9
1.588797612648995e9
1.588797613650615e9
1.588797614652015e9
1.588797615654104e9
1.588797616656221e9
1.588797617658723e9
1.588797618660876e9
1.588797619662546e9
1.588797620663744e9
1.588797621666169e9
1.588797622668357e9
1.588797623670544e9
1.588797624671739e9
1.588797625674223e9
1.588797626676652e9


In [6]:
delete!(ctx, pod);

## Specifying resource allocation

Optional, but recommended.

Helps the cluster to schedule processes better.
Safeguards your processes from being throttled or terminated.

In [7]:
julia_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "julia-pod",
               "namespace": "default",
               "labels": {
                   "name": "julia-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "julia",
                   "image": "julia:1.4.1",
                   "command": ["julia"],
                   "args": [
                       "-e",
                       "while true; println(time()); sleep(1); end"
                   ],
                   "resources": {
                       "limits": {
                           "cpu": "1"
                       },
                       "requests": {
                           "cpu": "500m"
                       }
                   }
               }]
           }
       }""");

In [8]:
pod = put!(ctx, julia_pod);

In [9]:
;kubectl logs julia-pod

1.588797669806216e9
1.588797670986561e9
1.588797671988748e9
1.58879767299078e9
1.588797673992975e9
1.588797674994933e9
1.588797675997072e9
1.588797676998646e9
1.58879767800087e9
1.588797679002619e9
1.588797680004936e9
1.588797681006953e9


In [10]:
delete!(ctx, pod);